In [ ]:
!pip install pandas pymongo
!pip install langchain-community

In [2]:
import os
import pandas as pd
from pymongo import MongoClient
from pymongo.errors import BulkWriteError

# --- CONFIGURATION ---
MONGO_URI = os.getenv("MONGO_URI", "mongodb://localhost:27017/")
DB_NAME = "genai_db"
COLLECTION_NAME = "product_data"
CSV_FILE = "sample_data.csv"  # This can be parameterized

def load_csv_to_mongodb(csv_path):
    try:
        df = pd.read_csv(csv_path)
        
        # Convert dataframe to dictionary
        data = df.to_dict(orient="records")
        print(f"📄 {len(data)} records read from CSV.")

        # Connect to MongoDB
        client = MongoClient(MONGO_URI)
        db = client[DB_NAME]
        collection = db[COLLECTION_NAME]

        # Optional: Drop previous collection for clean insert (for testing)
        collection.drop()

        # Insert data
        collection.insert_many(data)
        print(f"✅ Successfully inserted {len(data)} documents into MongoDB.")
    
    except BulkWriteError as bwe:
        print("⚠️ Bulk write error:", bwe.details)
    except Exception as e:
        print("❌ Error:", str(e))

if __name__ == "__main__":
    load_csv_to_mongodb(CSV_FILE)


📄 10 records read from CSV.
✅ Successfully inserted 10 documents into MongoDB.


In [ ]:
import requests
import ast
from pymongo import MongoClient

# Static system prompt
system_prompt ="""

You are a highly intelligent AI assistant specialized in converting natural language queries into precise MongoDB aggregation pipelines.

Your task is to generate ONLY a valid MongoDB aggregation pipeline as a Python list of dictionaries that matches the user's question exactly.

========================= SCHEMA =========================

Use exactly these field names (case-sensitive):

- _id
- ProductID
- ProductName
- Category
- Price
- Rating
- ReviewCount
- Stock
- Discount  → string with `%` (e.g., "10%")
- Brand
- LaunchDate → string in "dd-mm-yyyy"

========================= CHAIN-OF-THOUGHT INSTRUCTIONS =========================

For EVERY question, follow this step-by-step approach:

1. **Understand the question**:
   - Identify all filter conditions (date filters on LaunchDate, multiple categories, brands, price/rating/stock ranges, discounts).
   - Identify logical operators (AND, OR, NOT).
   - Identify sorting requirements (fields and ascending/descending).
   - Identify which fields to return (projection).
   - Identify if any numeric conversion or transformations are needed (e.g., discount strings like "15%" to numbers).
   - Identify if limits on number of documents are requested.

2. **Plan the pipeline stages**:
   - Use `$addFields` if any data conversion or new fields are needed (e.g., DiscountValue from Discount string).
   - Use a single `$match` stage combining all filters with `$and` or `$or`.
   - Use `$project` to return only requested fields and exclude `_id` if not needed.
   - Use `$sort` for sorting.
   - Use `$limit` if limiting the output count.

3. **Translate filters precisely**:
   - Date filters on `LaunchDate` are string comparisons (e.g., `"$gt": "01-01-2022"`).
   - Use `$in` for multiple categories or brands.
   - Use `$toInt` or `$toDouble` with `$substr` to convert Discount string percentages to numeric values.
   - Combine multiple conditions inside `$match` with `$and`.
   - Use `$gte`, `$lte`, `$gt`, `$lt` correctly for numeric filters.

4. **Build the pipeline JSON/Python list**:
   - Follow correct syntax and use exact field names.
   - No multiple `$match` stages; combine all filtering conditions in one.
   - Do not include extra fields in `$project` other than requested.

5. **Output only the aggregation pipeline** as a Python list of dictionaries — no explanation, markdown, or comments.

---

### Example 1:

**Question:**  
Find products by brands Apple or Samsung with rating at least 4.5, price between 100 and 500 inclusive, and discount more than 5%. Show only ProductName, Brand, Price, Rating, and Discount, sorted by Rating descending and limit results to 10.

**Chain-of-thought:**

- Filter Brand in ["Apple", "Samsung"].
- Filter Rating >= 4.5.
- Filter Price between 100 and 500 (inclusive).
- Convert Discount string to DiscountValue and filter DiscountValue > 5.
- Project requested fields only.
- Sort by Rating descending.
- Limit to 10 documents.

**Aggregation pipeline:**

[
  {
    "$addFields": {
      "DiscountValue": {
        "$toInt": {
          "$substr": ["$Discount", 0, { "$subtract": [ { "$strLenCP": "$Discount" }, 1 ] }]
        }
      }
    }
  },
  {
    "$match": {
      "$and": [
        { "Brand": { "$in": ["Apple", "Samsung"] } },
        { "Rating": { "$gte": 4.5 } },
        { "Price": { "$gte": 100, "$lte": 500 } },
        { "DiscountValue": { "$gt": 5 } }
      ]
    }
  },
  {
    "$project": {
      "ProductName": 1,
      "Brand": 1,
      "Price": 1,
      "Rating": 1,
      "Discount": 1,
      "_id": 0
    }
  },
  {
    "$sort": { "Rating": -1 }
  },
  {
    "$limit": 10
  }
]

---

### Example 2:

**Question:**  
List all products in categories Electronics or Sports that launched before 15-08-2021, have stock greater than 20, and review count greater than or equal to 50. Sort by LaunchDate ascending and show ProductName, Category, LaunchDate, Stock, and ReviewCount.

**Chain-of-thought:**

- Filter Category in ["Electronics", "Sports"].
- Filter LaunchDate < "15-08-2021" (string comparison).
- Filter Stock > 20.
- Filter ReviewCount >= 50.
- Project requested fields.
- Sort by LaunchDate ascending.

**Aggregation pipeline:**

[
  {
    "$match": {
      "$and": [
        { "Category": { "$in": ["Electronics", "Sports"] } },
        { "LaunchDate": { "$lt": "15-08-2021" } },
        { "Stock": { "$gt": 20 } },
        { "ReviewCount": { "$gte": 50 } }
      ]
    }
  },
  {
    "$project": {
      "ProductName": 1,
      "Category": 1,
      "LaunchDate": 1,
      "Stock": 1,
      "ReviewCount": 1,
      "_id": 0
    }
  },
  {
    "$sort": { "LaunchDate": 1 }
  }
]

---

### Now, process this input question exactly in the same way:


Input: {user_question}

Output:
"""

# Send prompt to local LLM
def get_query_from_llm(system_prompt: str, user_question: str) -> str:
    prompt = f"{system_prompt}\ninput: {user_question.strip()}\noutput:"
    response = requests.post(
        'http://localhost:11434/api/generate',
        json={
            'model': 'gemma:2b',
            'prompt': prompt,
            'stream': False
        }
    )
    data = response.json()
    query_code = data.get('response', '').strip()

    # Remove markdown if present
    if '```' in query_code:
        query_code = query_code.split('```')[1].replace('python', '').strip()
    
    return query_code

# Execute MongoDB aggregation
def run_query(pipeline_code: str, db_name="genai_db", collection_name="product_data"):
    try:
        pipeline = ast.literal_eval(pipeline_code)
        print("\n🧠 Aggregation Pipeline:\n", pipeline)

        client = MongoClient("mongodb://localhost:27017/")
        collection = client[db_name][collection_name]
        results = collection.aggregate(pipeline)

        print("\n📦 Query Results:")
        for doc in results:
            print(doc)
    except Exception as e:
        print("❌ Error during query execution:", e)

# Main logic
if __name__ == "__main__":
    user_question = input("Ask your MongoDB question:\n> ")
    print("\n🔁 Processing your question...,", user_question)
    if not user_question.strip():
        print("⚠️ Empty input.")
    else:
        pipeline_code = get_query_from_llm(system_prompt, user_question)
        if pipeline_code:
            run_query(pipeline_code)
        else:
            print("⚠️ No valid response from LLM.")

In [ ]:
import ast
import re
import requests
from pymongo import MongoClient

import json
import re

system_prompt = """

You are a highly intelligent AI assistant specialized in converting natural language queries into precise MongoDB aggregation pipelines.

Your task is to generate ONLY a valid MongoDB aggregation pipeline as a Python list of dictionaries that matches the user's question exactly.

========================= SCHEMA =========================

Use exactly these field names (case-sensitive):

- _id
- ProductID
- ProductName
- Category
- Price
- Rating
- ReviewCount
- Stock
- Discount  → string with `%` (e.g., "10%")
- Brand
- LaunchDate → string in "dd-mm-yyyy"

========================= CHAIN-OF-THOUGHT INSTRUCTIONS =========================

For EVERY question, follow this step-by-step approach:

1. **Understand the question**:
   - Identify all filter conditions (date filters on LaunchDate, multiple categories, brands, price/rating/stock ranges, discounts).
   - Identify logical operators (AND, OR, NOT).
   - Identify sorting requirements (fields and ascending/descending).
   - Identify which fields to return (projection).
   - Identify if any numeric conversion or transformations are needed (e.g., discount strings like "15%" to numbers).
   - Identify if limits on number of documents are requested.

2. **Plan the pipeline stages**:
   - Use `$addFields` if any data conversion or new fields are needed (e.g., DiscountValue from Discount string).
   - Use a single `$match` stage combining all filters with `$and` or `$or`.
   - Use `$project` to return only requested fields and exclude `_id` if not needed.
   - Use `$sort` for sorting.
   - Use `$limit` if limiting the output count.

3. **Translate filters precisely**:
   - Date filters on `LaunchDate` are string comparisons (e.g., `"$gt": "01-01-2022"`).
   - Use `$in` for multiple categories or brands.
   - Use `$toInt` or `$toDouble` with `$substr` to convert Discount string percentages to numeric values.
   - Combine multiple conditions inside `$match` with `$and`.
   - Use `$gte`, `$lte`, `$gt`, `$lt` correctly for numeric filters.

4. **Build the pipeline JSON/Python list**:
   - Follow correct syntax and use exact field names.
   - No multiple `$match` stages; combine all filtering conditions in one.
   - Do not include extra fields in `$project` other than requested.

5. **Output only the aggregation pipeline** as a Python list of dictionaries — no explanation, markdown, or comments.

---

### Example 1:

**Question:**  
Find products by brands Apple or Samsung with rating at least 4.5, price between 100 and 500 inclusive, and discount more than 5%. Show only ProductName, Brand, Price, Rating, and Discount, sorted by Rating descending and limit results to 10.

**Chain-of-thought:**

- Filter Brand in ["Apple", "Samsung"].
- Filter Rating >= 4.5.
- Filter Price between 100 and 500 (inclusive).
- Convert Discount string to DiscountValue and filter DiscountValue > 5.
- Project requested fields only.
- Sort by Rating descending.
- Limit to 10 documents.

**Aggregation pipeline:**

[
  {
    "$addFields": {
      "DiscountValue": {
        "$toInt": {
          "$substr": ["$Discount", 0, { "$subtract": [ { "$strLenCP": "$Discount" }, 1 ] }]
        }
      }
    }
  },
  {
    "$match": {
      "$and": [
        { "Brand": { "$in": ["Apple", "Samsung"] } },
        { "Rating": { "$gte": 4.5 } },
        { "Price": { "$gte": 100, "$lte": 500 } },
        { "DiscountValue": { "$gt": 5 } }
      ]
    }
  },
  {
    "$project": {
      "ProductName": 1,
      "Brand": 1,
      "Price": 1,
      "Rating": 1,
      "Discount": 1,
      "_id": 0
    }
  },
  {
    "$sort": { "Rating": -1 }
  },
  {
    "$limit": 10
  }
]

---

### Example 2:

**Question:**  
List all products in categories Electronics or Sports that launched before 15-08-2021, have stock greater than 20, and review count greater than or equal to 50. Sort by LaunchDate ascending and show ProductName, Category, LaunchDate, Stock, and ReviewCount.

**Chain-of-thought:**

- Filter Category in ["Electronics", "Sports"].
- Filter LaunchDate < "15-08-2021" (string comparison).
- Filter Stock > 20.
- Filter ReviewCount >= 50.
- Project requested fields.
- Sort by LaunchDate ascending.

**Aggregation pipeline:**

[
  {
    "$match": {
      "$and": [
        { "Category": { "$in": ["Electronics", "Sports"] } },
        { "LaunchDate": { "$lt": "15-08-2021" } },
        { "Stock": { "$gt": 20 } },
        { "ReviewCount": { "$gte": 50 } }
      ]
    }
  },
  {
    "$project": {
      "ProductName": 1,
      "Category": 1,
      "LaunchDate": 1,
      "Stock": 1,
      "ReviewCount": 1,
      "_id": 0
    }
  },
  {
    "$sort": { "LaunchDate": 1 }
  }
]

---
### Now, process this input question exactly in the same way:
Input: {user_question}

Output:
"""
def get_pipeline_from_llm(user_question: str) -> list:
    prompt = system_prompt.replace("{user_question}", user_question.strip())

    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "gemma:2b",
            "prompt": prompt,
            "stream": False
        }
    )
    response.raise_for_status()
    data = response.json()
    raw_text = data.get("response", "").strip()

    # Remove markdown fences if present
    clean_text = re.sub(r"```(?:python)?", "", raw_text).strip()

    try:
        # Parse as JSON instead of Python literals
        pipeline = json.loads(clean_text)
        return pipeline
    except Exception as e:
        print("❌ Failed to parse pipeline:", e)
        print("Response was:\n", clean_text)
        return None


def run_aggregation(pipeline):
    client = MongoClient("mongodb://localhost:27017/")
    collection = client["genai_db"]["product_data"]
    results = collection.aggregate(pipeline)
    return list(results)

def main():
    print("✅ Assistant initialized with schema memory.")
    while True:
        user_question = input("\nAsk your MongoDB question (or type 'exit' to quit):\n> ").strip()
        if user_question.lower() in ('exit', 'quit'):
            print("👋 Exiting assistant.")
            break

        print("\n🔁 Processing your question:", user_question)
        if not user_question:
            print("⚠️ Empty input.")
            continue

        pipeline = get_pipeline_from_llm(user_question)
        if not pipeline:
            print("⚠️ Could not get valid pipeline from LLM.")
            continue

        print("\n🧠 Aggregation Pipeline:")
        for stage in pipeline:
            print(stage)

        try:
            results = run_aggregation(pipeline)
            print("\n📦 Query Results:")
            for doc in results:
                print(doc)
        except Exception as e:
            print("❌ MongoDB query error:", e)

if __name__ == "__main__":
    main()



✅ Assistant initialized with schema memory.

🔁 Processing your question: Which products in the Electronics category have a rating of 4.5 or higher and are  in stock

🧠 Aggregation Pipeline:
{'$match': {'Category': 'Electronics', 'Rating': {'$gte': 4.5}}}

📦 Query Results:
{'_id': ObjectId('6837c8eaed1bfb470e8ff31a'), 'ProductID': 101, 'ProductName': 'Wireless Mouse', 'Category': 'Electronics', 'Price': 25.99, 'Rating': 4.5, 'ReviewCount': 200, 'Stock': 150, 'Discount': '10%', 'Brand': 'Logitech', 'LaunchDate': '15-01-2022'}
{'_id': ObjectId('6837c8eaed1bfb470e8ff31b'), 'ProductID': 102, 'ProductName': 'Gaming Keyboard', 'Category': 'Electronics', 'Price': 75.49, 'Rating': 4.7, 'ReviewCount': 350, 'Stock': 85, 'Discount': '5%', 'Brand': 'Corsair', 'LaunchDate': '20-11-2021'}
{'_id': ObjectId('6837c8eaed1bfb470e8ff31c'), 'ProductID': 103, 'ProductName': 'Noise Cancelling Headphones', 'Category': 'Electronics', 'Price': 199.99, 'Rating': 4.8, 'ReviewCount': 125, 'Stock': 60, 'Discount': '